# Example: Advanced Sudoku Optimization

The following notebook gives an extensive overview over the different possibilities that one has, when optimizing a configuration of checks and entries in a field design. For a more beginner-friendly guide on the basic optimization steps, check out the basic optimization tutorial, which can be found [here](https://github.com/janattig/SudokuPlantDesign.jl/blob/main/examples/sudoku_basic_optimization.ipynb).

This notebook is separated into sections as
1. [Creating an initial configuration](#1-Creating-an-initial-configuration)
2. [Showing information of a configuration](#2-Showing-information-of-a-configuration)
3. [Optimization: cost functions](#3-Optimization:-cost-functions)
4. [Optimization: updates](#4-Optimization:-updates)
5. [Optimization: additional information](#5-Optimization:-additional-information)

First, let us import the required modules

In [ ]:
using SudokuPlantDesign
using PyPlot

## 1 Creating an initial configuration

There are 4 key aspects when it comes to an initial configuration:
- [Creating a new object](#1.1-Creating-a-new-object)
- [Accessing individual plots](#1.2-Accessing-individual-plots)
- [Distributing missing plots](#1.3-Distributing-missing-plots)
- [Distributing checks and entries](#1.4-Distributing-checks-and-entries)

#### 1.1 Creating a new object

The command to creat a new configuration object is
```julia
get_configuration(
        blocksizes_x,
        blocksizes_y,
        number_of_check_types
        ;
        boundary_condition = :periodic
    )
```

where the parameters are as follows:
- `blocksizes_x` and `blocksizes_y` are lists which contain the sizes of blocks in the x and y direction respectively
- `number_of_check_types` denotes the number of different checks in the design
- `boundary_condition` is an optional parameter which denotes the type of boundary in the later optimization (when calculating distances). Possible options are `:open` or `:periodic`, however it can also be omitted

In [ ]:
# creates a configuration with 2 blocks in x-direction and 4 blocks in y-direction and 3 checks total
conf = get_configuration([9,9],[2,2,2,2],3);

#### 1.2 Accessing individual plots

A newly created objects starts as a plain design with only entries but no checks. One can set individual plots in the design to be either an *entry*, a *check* or an *empty (missing)* plot. The syntax is as follows:
```julia
set_entry!(conf, i,j)            # sets the plot on coordinates i,j to be an entry
set_entry!(conf, bi,bj, i,j)     # sets the plot on coordinates i,j in block bi,bj to be an entry

set_check!(conf, i,j, c)         # sets the plot on coordinates i,j to be check c
set_check!(conf, bi,bj, i,j, c)  # sets the plot on coordinates i,j in block bi,bj to be check c

set_empty!(conf, i,j)            # sets the plot on coordinates i,j to be missing (i.e. left empty)
set_empty!(conf, bi,bj, i,j)     # sets the plot on coordinates i,j in block bi,bj to be missing (i.e. left empty)
```

In [ ]:
# sets the plot at 3,2 to be check #2
set_check!(conf, 3,2, 2);

#### 1.3 Distributing missing plots

Of course it is tedious to set every plot by hand. Therefore, the function `empty_plots!` allows to designate an entire rectangular region in the design which is supposed to be empty (missing):
```julia
empty_plots!(
        conf, 
        range_x,
        range_y
    )
```
Here, `range_x` and `range_y` denote coordinate ranges in the format `from:to` of the region.

In [ ]:
# sets the region 1 <= x <= 1 and 3 <= y <= 4 to be missing plots
empty_plots!(conf, 1:1,3:4)

Another option is to use the function `empty_plots_in_block!` which allows to designate an entire block of the design as empty (missing):
```julia
empty_plots_in_block!(
        conf, 
        block_i, block_j
    )
```
Here, `block_i` and `block_j` specify the coordinates of the block.

In [ ]:
# sets the block 1,2 to be entirely missing
empty_plots_in_block!(conf, 1,2)

#### 1.4 Distributing checks and entries

It is also recommended to start the optimization of a design with some distribution of checks and entrys at hand. To initialize such a distribution, there are several possibilities implemented.

The function `initialize_entries!` allows to fill all non-empty plots of the design with checks, s.t. there is an exact number of entries remaining (which the breeder wants to distribute):
```julia
initialize_entries!(
        conf, 
        number_of_entries
    )
```

In [ ]:
# distributes exactly 119 entries (and leaves the rest as checks)
initialize_entries!(conf, 119)

Similarly, the function `initialize_checks!` allows to fill all non-empty plots of the design with random checks, s.t. there is an exact number of checks in total:
```julia
initialize_checks!(
        conf, 
        number_of_checks
    )
```

In [ ]:
# distributes exactly 42 random checks (and leaves the rest as entries)
initialize_checks!(conf, 42)

A third option is to distribute one check of every type per block with the function `initialize_checks_per_block!`
```julia
initialize_checks_per_block!(
        conf
    )
```

Note that this function distributes less checks if less space is available in a particular block (e.g. due to missing plots).

In [ ]:
# distributes exactly 1 check per type per block
initialize_checks_per_block!(conf)

## 2 Showing information of a configuration

Information about a configuration object can be accessed in two different ways: numeric information and plotting.

Numeric information can be printed by calling the function `print_info(conf)` as seen below

In [ ]:
# prints information on the current configuration conf
print_info(conf)

The configuration can also be plotted (utilizing `PyPlot` as the backend) with the following function:

```julia
show_configuration(
        conf
        ;
        zoom = 1.0,
        title_zoom = 1.0,
        cmap = "gist_rainbow",
        check_labels = true,
        dpi = 300,
        show_coordinates = false
    )
```
The several optional parameters are:
- `zoom` denotes the zoom of the entire figure
- `title_zoom` denotes a zoom that only acts on the figure title
- `cmap` is the color map used for plotting
- `check_labels` denotes wether there should be text labels in the check fields
- `dpi` the dpi of the figure, useful for exporting to `.png` files
- `show_coordinates` denotes wether a coordinate system should be drawn around the design

It should be noted, that plotting via `PyPlot` enables the user to use surrounding `PyPlot` syntax to refine or save the figure.

In [ ]:
show_configuration(conf, zoom=0.2)

## 3 Optimization: cost functions

To begin the optimization process, one has to specify a function which serves as the cost function of the optimization. The number it returns can be thought of as *how bad is the configuration*, i.e. high numbers are bad and low numbers are desirable.

There are several cost functions already implemented, which can be added together to create a new cost function as follows. These pre-implemented cost functions are the following (which can be found [here](https://github.com/janattig/SudokuPlantDesign.jl/blob/main/src/monte_carlo/cost_functions.jl)):

```julia
# cost functions penaltizing deviations from a fixed number of checks / entries
K_num_checks_total(conf, num_checks)
K_num_entries_total(conf, num_entries)

# cost functions penaltizing deviations of check numbers per type
K_num_checks_per_type(conf, num_checks)
K_num_checks_equal_per_type(conf)


# cost functions penaltizing check numbers per type and per block
K_min_checks_per_type_per_block(conf, num_checks)
K_max_checks_per_type_per_block(conf, num_checks)
K_checks_per_type_per_block(conf, num_checks)


# cost functions penaltizing check numbers per type and per row
K_min_checks_per_type_per_row(conf, num_checks)
K_max_checks_per_type_per_row(conf, num_checks)
K_checks_per_type_per_row(conf, num_checks)

# cost functions penaltizing check numbers per type and per colum
K_min_checks_per_type_per_column(conf, num_checks)
K_max_checks_per_type_per_column(conf, num_checks)
K_checks_per_type_per_column(conf, num_checks)


# cost functions penaltizing same checks being to close
K_neighbors_same_check_const(conf, cost)
K_neighbors_same_check_dmax_const(conf, dmax, cost)
K_neighbors_same_check_functional(conf, f)

# cost functions penaltizing different checks being to close
K_neighbors_different_check_const(conf, cost)
K_neighbors_different_check_dmax_const(conf, dmax, cost)
K_neighbors_different_check_functional(conf, f)
```

Note that chosing a correct cost function is vital to the success of the optimization procedure. At the same time, this process is going along with try and error. If you find your result lacking in some part, try to add a respective cost function which penaltizes this behavior.

In [ ]:
# combines different cost functions into a single cost function
cost_function(conf) =   K_checks_per_type_per_block(conf, 1) * 100 +
                        K_checks_per_type_per_column(conf, 1) * 10  +
                        K_checks_per_type_per_row(conf, 1) * 10 +
                        K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
                        K_neighbors_same_check_functional(conf, d->1/(d^3))

## 4 Optimization: updates

The second key ingredient in the optimization process are updates. These can be thought of as the step on *how to produce a new configuration from an old one*. In optimizing, the algorithm produces many updates and only follows those which the cost function deems *good updates*.

There are several different updates implemented into the code, which are the following:
- `UpdateInsertCheck()` and `UpdateInsertCheck()` are used to add and remove checks to the configuration
- `UpdateNewCheckLabel()` gives a random check a new label, i.e. changes its number
- `UpdateSwapCheckCheck()` swaps two checks in the configuration with each other
- `UpdateSwapCheckEntry()` swaps a check and an entry in the configuration with each other

Note that apart from `UpdateInsertCheck()` and `UpdateInsertCheck()`, all updates *preserve* the total number of checks and therefore can be used when the number of entries is fixed, even without specification in the cost function.

In [ ]:
# select a list of updates for the algorithm to choose from
updates = [
    UpdateNewCheckLabel(),
    UpdateSwapCheckCheck(),
    UpdateSwapCheckEntry()
]

## 5 Optimization: additional information

With updates and cost function defined, the optimization can be run with the following command:
```julia
optimize_design!(
        conf,
        updates,
        cost_function,
        number_of_updates = 100000
    )
```
The function performs the specified number of updates and returns an array containing the cost values after each step (for later analysis).

In [ ]:
costs = optimize_design!(
    conf,
    updates,
    cost_function,
    500000
);